### HTSeq - Install it in linux-based system
HTSeq (https://htseq.readthedocs.io) is an alternative library that's used for processing NGS data. Most of the functionality made available by HTSeq is actually available in other libraries covered in this book, but you should be aware of it as an alternative way of processing NGS data. HTSeq supports, among others, FASTA, FASTQ, SAM (via pysam), VCF, GFF, and Browser Extensible Data (BED) file formats. It also includes a set of abstractions for processing (mapped) genomic data, encompassing concepts like genomic positions and intervals or alignments. A complete examination of the features of this library is beyond our scope, so we will concentrate on a small subset of features. We will take this opportunity to also introduce the BED file format.

The BED format allows for the specification of features for annotations tracks. It has many uses, but it's common to load BED files into genome browsers to visualize features. Each line includes information about at least the position (chromosome, start and end) and also optional fields such as name or strand. Full details about the format can be found at https://genome.ucsc.edu/FAQ/FAQformat.html#format1.

In [1]:
from collections import defaultdict
import re
import HTSeq

Our simple example will use data from the region where the LCT gene is located in the human genome. The LCT gene codifies lactase, an enzyme involved in the digestion of lactose.

### 1. Read the file

In [2]:
lct_bed = HTSeq.BED_Reader('LCT.bed')

### 2. Extract all the types of features via its name 

In [3]:
feature_types = defaultdict(int)

for rec in lct_bed:
    last_rec = rec
    feature_types[re.search('([A-Z]+)', rec.name).group(0)] += 1

print(feature_types)

#Code specific to this dataset, document

defaultdict(<class 'int'>, {'ENSE': 27, 'NM': 17, 'CCDS': 17})


### 3. Store the last record so that we can inspect it

In [4]:
print(last_rec)
print(last_rec.name)
print(type(last_rec))
interval = last_rec.iv
print(interval)
print(type(interval))

<GenomicFeature: BED line 'CCDS2178.11' at 2: 135788543 -> 135788322 (strand '-')>
CCDS2178.11
<class 'HTSeq.features.GenomicFeature'>
2:[135788323,135788544)/-
<class 'HTSeq._HTSeq.GenomicInterval'>


### 4. Dig deeper into the interval

In [5]:
print(interval.chrom, interval.start, interval.end)
print(interval.strand)
print(interval.length)
print(interval.start_d)
print(interval.start_as_pos)
print(type(interval.start_as_pos))

#talk about overlaps


2 135788323 135788544
-
221
135788543
2:135788323/-
<class 'HTSeq._HTSeq.GenomicPosition'>


### 5. Extract some statistics from our coding regions (CCDS records)

In [6]:
exon_start = None
exon_end = None
sizes = []
for rec in lct_bed:
    if not rec.name.startswith('CCDS'):
        continue
    interval = rec.iv
    exon_start = min(interval.start, exon_start or interval.start)
    exon_end = max(interval.length, exon_end or interval.end)
    sizes.append(interval.length)
sizes.sort()
print("Num exons: %d / Begin: %d / End %d" % (len(sizes), exon_start, exon_end))
print("Smaller exon: %d / Larger exon: %d / Mean size: %.1f" % (sizes[0], sizes[-1], sum(sizes)/len(sizes)))

Num exons: 17 / Begin: 135788323 / End 135837169
Smaller exon: 79 / Larger exon: 1551 / Mean size: 340.2
